In [2]:
# Installar pip isntall psycopg2-binary
import psycopg2
import pandas as pd

# Insertar csv a postgresql


In [14]:
data = pd.read_csv('./scrap_data_final.csv', encoding='latin-1',sep = ';')

data.head()


,production_family,wk,fecha,fa_pn,pn_componente,wo,quantity,cod_defecto,label,linea,turno
0,Wands (Kamuk),Week 01,1/3/2020,48400,48404,2046690.0,1,D1093,1,Line 13,T1
1,Wands (Kamuk),Week 01,1/3/2020,48400,48414,2046690.0,1,DO085,1,Line 13,T 1
2,Wands (Kamuk),Week 01,1/3/2020,48400,48416,2046690.0,1,DO1646,1,Line 13,T 1
3,Wands (Kamuk),Week 01,1/3/2020,48400,49697,2046690.0,4,DO1644,1,Line 13,T 1
4,Wands (Kamuk),Week 01,1/3/2020,48400,20363,2046690.0,4,NaN,0,Line 13,T 1


In [15]:
data.dtypes

production_family     object
wk                    object
fecha                 object
fa_pn                 object
pn_componente         object
wo                   float64
quantity               int64
cod_defecto           object
label                  int64
linea                 object
turno                 object
dtype: object

# Correr desde aqui

In [16]:
create_query = 'DROP TABLE IF EXISTS bigdata.scrap;\
         CREATE TABLE bigdata.scrap\
        (\
        production_family varchar(256),wk varchar(256), fecha varchar(256), fa_pn varchar(256), pn_componente varchar(256), wo varchar(256),\
       quantity float, cod_defecto varchar(256), label integer, linea varchar(256), turno varchar(256)\
    ); \
commit;' 
create_query

'DROP TABLE IF EXISTS bigdata.scrap;         CREATE TABLE bigdata.scrap        (        production_family varchar(256),wk varchar(256), fecha varchar(256), fa_pn varchar(256), pn_componente varchar(256), wo varchar(256),       quantity float, cod_defecto varchar(256), label integer, linea varchar(256), turno varchar(256)    ); commit;'

In [10]:
con = psycopg2.connect( host='localhost', user='postgres', password='root', dbname='postgres', port = 5432 )
cursor=con.cursor()

In [12]:
cursor.execute('delete from bigdata.scrap; commit;')

In [17]:
cursor.execute(create_query)

In [18]:
pd.read_sql('select * from bigdata.scrap', con)


,production_family,wk,fecha,fa_pn,pn_componente,wo,quantity,cod_defecto,label,linea,turno


In [20]:
with open('scrap_data_final.csv', 'r') as f:
# Notice that we don't need the `csv` module.
    next(f) # Skip the header row.
    cursor.copy_from(f, 'bigdata.scrap', sep=';')
    con.commit()


In [21]:
pd.read_sql('select * from bigdata.scrap', con)

,production_family,wk,fecha,fa_pn,pn_componente,wo,quantity,cod_defecto,label,linea,turno
0,Wands (Kamuk),Week 01,1/3/2020,48400,48404,2046690.0,1.0,D1093,1,Line 13,T1
1,Wands (Kamuk),Week 01,1/3/2020,48400,48414,2046690.0,1.0,DO085,1,Line 13,T 1
2,Wands (Kamuk),Week 01,1/3/2020,48400,48416,2046690.0,1.0,DO1646,1,Line 13,T 1
3,Wands (Kamuk),Week 01,1/3/2020,48400,49697,2046690.0,4.0,DO1644,1,Line 13,T 1
4,Wands (Kamuk),Week 01,1/3/2020,48400,20363,2046690.0,4.0,,0,Line 13,T 1
...,...,...,...,...,...,...,...,...,...,...,...
19922,SA,Week 04,1/26/2020,16358,7229,2049249.0,3.0,DW07,1,Returns,T 1
19923,SA,Week 04,1/26/2020,93846,42003,2048805.0,8.0,DW01,1,SA,T 1
19924,SA,Week 04,1/26/2020,93846,4496,2048805.0,6.0,,0,SA,T 1
19925,SA,Week 04,1/26/2020,93846,12533,2048805.0,6.0,,0,SA,T 1


In [22]:
con.close()

# Aqui ya se conecta con pyspark

In [1]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", "./postgresql-42.2.8.jar") \
    .config("spark.executor.extraClassPath", "./postgresql-42.2.8.jar") \
    .getOrCreate()

# Reading single DataFrame in Spark by retrieving all rows from a DB table.
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/") \
    .option("user", "postgres") \
    .option("password", "root") \
    .option("dbtable", "bigdata.scrap")\
    .load()

In [14]:
df.show()

+-----------------+-------+--------+-----+-------------+---------+--------+-----------+--------------------+-------+-----+
|production_family|     wk|   fecha|fa_pn|pn_componente|       wo|quantity|cod_defecto|               causa|  linea|turno|
+-----------------+-------+--------+-----+-------------+---------+--------+-----------+--------------------+-------+-----+
|    Wands (Kamuk)|Week 01|1/3/2020|48400|        48404|2046690.0|     1.0|      D1093|          C7-Proceso|Line 13|   T1|
|    Wands (Kamuk)|Week 01|1/3/2020|48400|        48414|2046690.0|     1.0|      DO085|          C7-Proceso|Line 13|  T 1|
|    Wands (Kamuk)|Week 01|1/3/2020|48400|        48416|2046690.0|     1.0|     DO1646|          C7-Proceso|Line 13|  T 1|
|    Wands (Kamuk)|Week 01|1/3/2020|48400|        49697|2046690.0|     4.0|     DO1644|          C7-Proceso|Line 13|  T 1|
|    Wands (Kamuk)|Week 01|1/3/2020|48400|        20363|2046690.0|     4.0|           |C17- Pruebas de c...|Line 13|  T 1|
|    Wands (Kamu

# Cargo los archivos, si postgres 

# Estudio de Scrap: Empresa Smith and Nephew 2020

Con la finalidad de entender el comportamiento del scrap en del 2020, la empresa Smith and Nephew CR. Establecio un recuento del scrap por modo de falla. Dado que la cantidad de fallos que generan scrap es muy amplia, se priorizo determinar el scrap producido por fallos de proceso, dado que el historico de datos del 2019 mostro todos los fallos de proceso se debian por fallos en la validacion de proceso. 
La correccion de este tipo de fallos implican una amplia inversion de la empresa, por lo que es de interese predecir cuando una familia de productos genera fallos por proceso, al la vez que se relaciona el impacto de los turnos y la linea en cada fallo. 

El set de datos obtenidos para el proyecto equivale a todo el desecho realizado en la empresa por linea durante el mes de enero del 2020, como parte del proyecto piloto para poder predecir desgastes en las condiciones validadas en las lineas de producion. 

## Informacion en el set de datos: 
- production_family: Familia de productos, los productos se dividen segun su mercado y este es un indicador de la ganancia final.
- wk: Semana en la que se documento el scrap, la produccion se planea por semanas. Categorico 
- fecha: fecha en la que se registro el scrap. Categorico
- fa_pn: Numero de parte del producto final en el que se detecto el scrap.Categorico 
- pn_componente: Numero de parte del componente desechado.Categorico 
- wo: Order de produccion en la cual se detecto el scrap.Categorico 
- quantity: cantidad de unidades desechadas. Numerico
- cod_defecto: condigo de defecto registrado.Categorico 
- label: Es la causa de fallo, para este estudio piloto se designo como label. 1 es un fallo por proceso, 0 cualquier otro fallo. Categorico ( Trasformado en binario)
- linea: de produccion en la que se realizo el fallo. Categorico 
- turno: Turno de produccion, con o sin extras en el que se registro el fallo.Categorico 


In [3]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

spark = SparkSession \
    .builder \
    .appName("scrap_data") \
    .config("spark.driver.extraClassPath", "./postgresql-42.2.8.jar") \
    .config("spark.executor.extraClassPath", "./postgresql-42.2.8.jar") \
    .getOrCreate()

# Reading single DataFrame in Spark by retrieving all rows from a DB table.
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/") \
    .option("user", "postgres") \
    .option("password", "root") \
    .option("dbtable", "bigdata.scrap")\
    .load()

# Exploracion de los datos.

Cuantos datos se tienen y el numero de columnas.

In [4]:
#check the shape of the data 
print((df.count(),len(df.columns)))

(19927, 11)


El esquema del set de datos

In [5]:
#printSchema
df.printSchema()

root
 |-- production_family: string (nullable = true)
 |-- wk: string (nullable = true)
 |-- fecha: string (nullable = true)
 |-- fa_pn: string (nullable = true)
 |-- pn_componente: string (nullable = true)
 |-- wo: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- cod_defecto: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- linea: string (nullable = true)
 |-- turno: string (nullable = true)



Identificacion de las calumnas 

In [176]:
#number of columns in dataset
df.columns

['production_family',
 'wk',
 'fecha',
 'fa_pn',
 'pn_componente',
 'wo',
 'quantity',
 'cod_defecto',
 'label',
 'linea',
 'turno']

Visualizacion del set de datos, las primeras 5 lineas.

In [177]:
df.limit(5).toPandas().head()

,production_family,wk,fecha,fa_pn,pn_componente,wo,quantity,cod_defecto,label,linea,turno
0,Wands (Kamuk),Week 01,1/3/2020,48400,48404,2046690.0,1,D1093,1,Line 13,T1
1,Wands (Kamuk),Week 01,1/3/2020,48400,48414,2046690.0,1,DO085,1,Line 13,T 1
2,Wands (Kamuk),Week 01,1/3/2020,48400,48416,2046690.0,1,DO1646,1,Line 13,T 1
3,Wands (Kamuk),Week 01,1/3/2020,48400,49697,2046690.0,4,DO1644,1,Line 13,T 1
4,Wands (Kamuk),Week 01,1/3/2020,48400,20363,2046690.0,4,None,0,Line 13,T 1


In [178]:
#Exploratory Data Analysis
df.describe().show()

+-------+-----------------+-------+---------+--------------------+--------------------+------------------+------------------+-----------+------------------+-------------+-----+
|summary|production_family|     wk|    fecha|               fa_pn|       pn_componente|                wo|          quantity|cod_defecto|             label|        linea|turno|
+-------+-----------------+-------+---------+--------------------+--------------------+------------------+------------------+-----------+------------------+-------------+-----+
|  count|            19927|  19927|    19927|               19918|               19855|             19912|             19927|      11476|             19927|        19927|19927|
|   mean|             null|   null|     null|1.5052684017250178E7|3.0355238520430107E7|2065847.6619124147|17.262457971596326|       null|0.3141466352185477|         null| null|
| stddev|             null|   null|     null|2.9612881848654352E7| 3.069515495343108E8| 598355.0758189503|158.75686

Identificacion del total de fallos encontrados por proceso

In [179]:
df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 6260|
|    0|13667|
+-----+-----+



Que la causa de scrappor falla de processo (1) es casi una tercera parter de los datos, la cual implica que las linea tienen muchos fallos en el proceso validado. Este tipo de fallos produce grandes perdidas operativas. El objetivo del estidio es predecir cuando va a fallar una familia de productos por una condicion de validacion.

## Variables a considerar en el estudio.

El alcance selecionado para este estudio es analizar el impacto del scrap por proceso por familia de productos, turno y linea. Esto debido a la cantidad de datos, posterior a este estudio se recopilaran datos para unir el impacto por componente y por producto.

In [180]:
#Contar las categorias de familias de productos
df.groupBy('production_family').count().show()

+-----------------+-----+
|production_family|count|
+-----------------+-----+
|             Opus|10658|
|           Final | 3891|
|               SA|   24|
|              PKG| 1591|
|    Wands (Kamuk)| 3035|
|      Rapid Rhino|  728|
+-----------------+-----+



In [181]:
#Contar las categorias de turno
df.groupBy('turno').count().show()

+-----+-----+
|turno|count|
+-----+-----+
|   T3|  222|
|  T 1|10912|
|   T1|    3|
|  T 2| 8790|
+-----+-----+



In [182]:
#Contar las categorias de turno
df.groupBy('Linea').count().show()

+----------------+-----+
|           Linea|count|
+----------------+-----+
|   Line 6 (Evac)| 1723|
|       Multiwire|   50|
|         Line 13|  735|
|Line 17 (Vulcan)|  174|
|         Returns|   66|
|         Line 14|  424|
|            Flow|    2|
|  Line 16 (Echo)|  972|
|              SA|  278|
|         Ambient|   25|
|   LINE 11 (EPX)|  728|
|             PKG| 1591|
|          Linea9| 1919|
|         Line 15|  742|
|  Line 3 (Pilot)|  541|
|           Laser|  210|
| Line 7 (Reflex)| 1176|
|    Line 2 (STV)|  523|
|          Line 4| 2136|
|   Line 5 (SMX2)| 1498|
+----------------+-----+
only showing top 20 rows



Cada condicion de turno es diferente: Para efectos del estudio T3 es turno nocturno, T 1 es turno 1 con gente en extras, T1 sin gente en extras, T 2 es turno 2 con gente en extras y T2 con gente sin extras, en el periodo que se esta estudiando en el turno 2 no hay extras. Los turnos igualmente son un atributo. 

### Transformar los datos 

Dado que la causa es la salida y es un atributo,  familia de productos, linea y turno es un atributo, y son los  feature de relevancia se requiere una trasnformacion de tipo One Hot Encoder:

In [183]:
#import required libraries
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

In [184]:
turno_indexer = StringIndexer(inputCol="turno", outputCol="turno_index").fit(df)
df = turno_indexer.transform(df)
turno_encoder = OneHotEncoder(inputCol="turno_index", outputCol="turno_purpose_vec")
df = turno_encoder.transform(df)

In [185]:
df.select(['turno','turno_index','turno_purpose_vec']).show(20,False)

+-----+-----------+-----------------+
|turno|turno_index|turno_purpose_vec|
+-----+-----------+-----------------+
|T1   |3.0        |(3,[],[])        |
|T 1  |0.0        |(3,[0],[1.0])    |
|T 1  |0.0        |(3,[0],[1.0])    |
|T 1  |0.0        |(3,[0],[1.0])    |
|T 1  |0.0        |(3,[0],[1.0])    |
|T 1  |0.0        |(3,[0],[1.0])    |
|T 1  |0.0        |(3,[0],[1.0])    |
|T 1  |0.0        |(3,[0],[1.0])    |
|T 1  |0.0        |(3,[0],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
|T 2  |1.0        |(3,[1],[1.0])    |
+-----+-----------+-----------------+
only showing top 20 rows



In [186]:
production_family_indexer = StringIndexer(inputCol="production_family", outputCol="production_family_index").fit(df)
df = production_family_indexer.transform(df)
production_family_encoder = OneHotEncoder(inputCol="production_family_index", outputCol="production_family_purpose_vec")
df = production_family_encoder.transform(df)

In [187]:
df.select(['production_family','production_family_index','production_family_purpose_vec']).show(20,False)

+-----------------+-----------------------+-----------------------------+
|production_family|production_family_index|production_family_purpose_vec|
+-----------------+-----------------------+-----------------------------+
|Wands (Kamuk)    |2.0                    |(5,[2],[1.0])                |
|Wands (Kamuk)    |2.0                    |(5,[2],[1.0])                |
|Wands (Kamuk)    |2.0                    |(5,[2],[1.0])                |
|Wands (Kamuk)    |2.0                    |(5,[2],[1.0])                |
|Wands (Kamuk)    |2.0                    |(5,[2],[1.0])                |
|Wands (Kamuk)    |2.0                    |(5,[2],[1.0])                |
|Wands (Kamuk)    |2.0                    |(5,[2],[1.0])                |
|Wands (Kamuk)    |2.0                    |(5,[2],[1.0])                |
|Wands (Kamuk)    |2.0                    |(5,[2],[1.0])                |
|Wands (Kamuk)    |2.0                    |(5,[2],[1.0])                |
|Wands (Kamuk)    |2.0                

In [188]:
linea_indexer = StringIndexer(inputCol="linea", outputCol="linea_index").fit(df)
df = linea_indexer.transform(df)
linea_encoder = OneHotEncoder(inputCol="linea_index", outputCol="linea_purpose_vec")
df = linea_encoder.transform(df)

In [189]:
df.select(['linea','linea_index','linea_purpose_vec']).show(20,False)

+-------+-----------+-----------------+
|linea  |linea_index|linea_purpose_vec|
+-------+-----------+-----------------+
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
|Line 13|10.0       |(24,[10],[1.0])  |
+-------+-----------+-----------------+
only showing top 20 rows



Ahora usamos "VectorAssembler" para crear un único vector de features y clase, para ser usado por nuestro modelo de entrenamiento:
   

In [190]:
from pyspark.ml.feature import VectorAssembler

In [191]:
df.columns

['production_family',
 'wk',
 'fecha',
 'fa_pn',
 'pn_componente',
 'wo',
 'quantity',
 'cod_defecto',
 'label',
 'linea',
 'turno',
 'turno_index',
 'turno_purpose_vec',
 'production_family_index',
 'production_family_purpose_vec',
 'linea_index',
 'linea_purpose_vec']

In [192]:
df_assembler = VectorAssembler(inputCols=['turno_purpose_vec','production_family_purpose_vec', 'linea_purpose_vec'], outputCol="features")
df = df_assembler.transform(df)

In [193]:
df.printSchema()

root
 |-- production_family: string (nullable = true)
 |-- wk: string (nullable = true)
 |-- fecha: string (nullable = true)
 |-- fa_pn: string (nullable = true)
 |-- pn_componente: string (nullable = true)
 |-- wo: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- cod_defecto: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- linea: string (nullable = true)
 |-- turno: string (nullable = true)
 |-- turno_index: double (nullable = false)
 |-- turno_purpose_vec: vector (nullable = true)
 |-- production_family_index: double (nullable = false)
 |-- production_family_purpose_vec: vector (nullable = true)
 |-- linea_index: double (nullable = false)
 |-- linea_purpose_vec: vector (nullable = true)
 |-- features: vector (nullable = true)



In [195]:
df.select(['features','label']).show(20,False)

+---------------------------+-----+
|features                   |label|
+---------------------------+-----+
|(32,[5,18],[1.0,1.0])      |1    |
|(32,[0,5,18],[1.0,1.0,1.0])|1    |
|(32,[0,5,18],[1.0,1.0,1.0])|1    |
|(32,[0,5,18],[1.0,1.0,1.0])|1    |
|(32,[0,5,18],[1.0,1.0,1.0])|0    |
|(32,[0,5,18],[1.0,1.0,1.0])|0    |
|(32,[0,5,18],[1.0,1.0,1.0])|0    |
|(32,[0,5,18],[1.0,1.0,1.0])|1    |
|(32,[0,5,18],[1.0,1.0,1.0])|1    |
|(32,[1,5,18],[1.0,1.0,1.0])|0    |
|(32,[1,5,18],[1.0,1.0,1.0])|0    |
|(32,[1,5,18],[1.0,1.0,1.0])|1    |
|(32,[1,5,18],[1.0,1.0,1.0])|0    |
|(32,[1,5,18],[1.0,1.0,1.0])|1    |
|(32,[1,5,18],[1.0,1.0,1.0])|0    |
|(32,[1,5,18],[1.0,1.0,1.0])|0    |
|(32,[1,5,18],[1.0,1.0,1.0])|1    |
|(32,[1,5,18],[1.0,1.0,1.0])|0    |
|(32,[1,5,18],[1.0,1.0,1.0])|0    |
|(32,[1,5,18],[1.0,1.0,1.0])|1    |
+---------------------------+-----+
only showing top 20 rows



## Construir el modelo

In [211]:
model_df=df.select(['features','label'])

In [212]:
from pyspark.ml.classification import LogisticRegression

In [213]:
#split the data 
training_df,test_df = model_df.randomSplit([0.75,0.25])

In [214]:
training_df.count()

14940

In [215]:
training_df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 4719|
|    0|10221|
+-----+-----+



In [216]:
test_df.count()

4987

In [217]:
test_df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 1541|
|    0| 3446|
+-----+-----+



# Regresión Logística.

La regresión logística es un tipo de análisis de regresión utilizado para predecir el resultado de una variable categórica (una variable que puede adoptar un número limitado de categorías)  en función de las variables independientes o predictoras. Dado que el tipo de defecto es una varible categorica (Proceso, paro de maquina, error de operario...) lo mas optimo es iniciar con esta regresion en primera instancia.
 
## Entrenamiento

In [218]:
log_reg=LogisticRegression().fit(training_df)

In [219]:
#Training Results

In [220]:
lr_summary=log_reg.summary

In [221]:
lr_summary.accuracy

0.7151271753681392

In [222]:
lr_summary.areaUnderROC

0.7675459441075684

In [223]:
print(lr_summary.precisionByLabel)

[0.7136614370656924, 0.7359836901121305]


In [225]:
print(lr_summary.recallByLabel)

[0.9746600136972899, 0.15299851663488026]


In [226]:
predictions = log_reg.transform(test_df)
predictions.show(10)


+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(32,[0,3],[1.0,1.0])|    0|[0.99931135577201...|[0.73092316141517...|       0.0|
|(32,[0,3],[1.0,1.0])|    1|[0.99931135577201...|[0.73092316141517...|       0.0|
|(32,[0,3,8],[1.0,...|    0|[4.67003104270878...|[0.99071503985679...|       0.0|
|(32,[0,3,8],[1.0,...|    0|[4.67003104270878...|[0.99071503985679...|       0.0|
|(32,[0,3,8],[1.0,...|    0|[4.67003104270878...|[0.99071503985679...|       0.0|
|(32,[0,3,8],[1.0,...|    0|[4.67003104270878...|[0.99071503985679...|       0.0|
|(32,[0,3,8],[1.0,...|    0|[4.67003104270878...|[0.99071503985679...|       0.0|
|(32,[0,3,8],[1.0,...|    0|[4.67003104270878...|[0.99071503985679...|       0.0|
|(32,[0,3,8],[1.0,...|    0|[4.67003104270878...|[0.99071503985679...|       0.0|
|(32,[0,3,8],[1.

## Testing

In [227]:
model_predictions = log_reg.transform(test_df)


In [228]:
model_predictions = log_reg.evaluate(test_df)

In [229]:
model_predictions.accuracy

0.7166633246440746

In [230]:
model_predictions.weightedPrecision

0.7126280106025605

In [231]:
model_predictions.recallByLabel

[0.9724318049912942, 0.14471122647631407]

In [232]:
print(model_predictions.precisionByLabel)

[0.717712572285286, 0.7012578616352201]


In [233]:
model_predictions.areaUnderROC

0.7595613494263774

El testing predice en un 0.7136614370656924  mientras que el modelo predice en un 0.717712572285286 por lo que el modelo se considera optimo para el estudio en cuestion. 

# Random Forest

Dado que el metodo de random forest es muy poderoso con datos con multiples categorias, por lo que se espera un buen resultado con su aplicacion.

In [234]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(training_df)

In [235]:
model_predictions = rf_model.transform(test_df)

En los parámetros a probar se selecciona **maxDepth**, **maxBins** y **numTrees**, se aplica cross-validation para determinar el mejor modelo y se usa además five-fold cross-validation (4 partes para entrenamiento y 1 para testing).

### Hiperparametros utilizados
- Para iniciar se van a utilizar los mismos parametros de referencia utilizados en el ejemplo del curos: 
        maxDepth [5,10,20,25,30] , maxBins [20,30,40 ] y numTrees [5, 20,50]

In [236]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import time


start = time.time()

evaluator = BinaryClassificationEvaluator()

rf = RandomForestClassifier()
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [5,10,20,25,30])
             .addGrid(rf.maxBins, [20,30,40 ])
             .addGrid(rf.numTrees, [5, 20,50])
             .build())
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cv_model = cv.fit(training_df)

end = time.time()
m = (end - start)/60
m = round(m, 2)
s = (end - start)
s = round(s, 2)
print(f"Time: {m} min / {s} sec")

Time: 10.38 min / 622.96 sec


In [237]:
best_rf_model = cv_model.bestModel

## Testing 

In [238]:
# Generate predictions for entire dataset
model_predictions = best_rf_model.transform(test_df)

In [239]:
true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
actual_pos=model_predictions.filter(model_predictions['label']==1).count()
pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()

In [240]:
#Recall 
float(true_pos)/(actual_pos)

0.14341336794289422

El Recall de una clase expresa cuan bien puede el modelo detectar a esa clase. Por lo que un 14% no es un buen valor, es decir que puede diferenciar entre otros fallos y el fallo por validacion en un 14%.

In [241]:
#Precision on test Data 
float(true_pos)/(pred_pos)

0.7015873015873015

La Precisión de una clase define cuan confiable es un modelo en responder si un punto pertenece a esa clase. Un 70% es un buen valor.

## Conclusion

Al obtenerse un recall muy bajo, se buscara optimizar el modelo para mejorar la prediccion.

In [242]:
model_predictions.limit(20).toPandas().head(20)

,features,label,rawPrediction,probability,prediction
0,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[26.479887605827507, 23.520112394172493]","[0.5295977521165501, 0.47040224788344986]",0.0
1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[26.479887605827507, 23.520112394172493]","[0.5295977521165501, 0.47040224788344986]",0.0
2,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
3,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
4,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
5,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
6,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
7,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
8,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
9,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0


### Hiperparametros utilizados modelo 2
- Se ajustan a un modelo mas robusto para mejorar el recall
        maxDepth [2,4,6,8,10,15,20,25,30] , maxBins [5,10,15,20,25,30,35,40 ] y numTrees [5, 10, 20,30,50]

In [244]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import time


start = time.time()

evaluator = BinaryClassificationEvaluator()

rf = RandomForestClassifier()
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2,4,6,8,10,15,20,25,30])
             .addGrid(rf.maxBins, [5,10,15,20,25,30,35,40 ])
             .addGrid(rf.numTrees, [5, 10, 20,30,50])
             .build())
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cv_model = cv.fit(training_df)

end = time.time()
m = (end - start)/60
m = round(m, 2)
s = (end - start)
s = round(s, 2)
print(f"Time: {m} min / {s} sec")

Time: 59.67 min / 3580.48 sec


In [245]:
best2_rf_model = cv_model.bestModel

## Testing

In [246]:
# Generate predictions for entire dataset
model_predictions = best2_rf_model.transform(test_df)

In [247]:
true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
actual_pos=model_predictions.filter(model_predictions['label']==1).count()
pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()

In [248]:
#Recall2 
float(true_pos)/(actual_pos)

0.14341336794289422

Se mantuvo el 14% en el recall, esto implica que este modelo no va a converger con los datos analizados.

In [249]:
#Precision on test Data 
float(true_pos)/(pred_pos)

0.7015873015873015

In [250]:
model_predictions.limit(20).toPandas().head(20)

,features,label,rawPrediction,probability,prediction
0,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[26.479887605827507, 23.520112394172493]","[0.5295977521165501, 0.47040224788344986]",0.0
1,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[26.479887605827507, 23.520112394172493]","[0.5295977521165501, 0.47040224788344986]",0.0
2,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
3,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
4,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
5,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
6,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
7,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
8,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0
9,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[49.15889353777858, 0.8411064622214095]","[0.9831778707555717, 0.01682212924442819]",0.0


# Conclusiones

El modelo mas adecuado para este estudio es una regresion logistica por los valores obtenidos durante la evaluacion del modelo. 